# 刚体变换练习题

In [21]:
# python libraries
import numpy as np
from pydrake.all import RigidTransform, RotationMatrix

# 问题描述

在课堂上，我们学习了空间变换的基础知识。本练习将应用你在课堂上学到的规则来计算简单的刚体变换。

**本练习的主要步骤如下：**

1. 计算不同参考系下的刚体变换。

2. 利用空间变换设计抓取位姿。

# 刚体变换练习

简要回顾一下，我们在[课堂](http://manipulation.csail.mit.edu/pick.html#spatial_algebra)上介绍了空间变换的两条规则。

$${^AX^B} {^BX^C} = {^AX^C},$$

$$[^AX^B]^{-1} = {^BX^A}.$$

请注意，变换的规则基于下述位置和旋转的变换规则。

同一坐标系下位置的加法：

$$^Ap^B_F + ^Bp^C_F = ^Ap^C_F.$$

加法逆元：

$$^Ap^B_F = - ^Bp^A_F.$$

点的旋转：

$$^Ap^B_G = {^GR^F} ^Ap^B_F.$$

旋转的链式法则：

$${^AR^B} {^BR^C} = {^AR^C}.$$

旋转的逆：

$$[^AR^B]^{-1} = {^BR^A}.$$

应用这些规则将得到与前述两条规则相同的结果。

在Drake中，你可以通过如下方式相乘刚体变换：

```python

X_AB.multiply(X_BC)

X_AB @ X_BC

```

你也可以通过[inverse](https://drake.mit.edu/pydrake/pydrake.math.html?highlight=rigidtransform#pydrake.math.RigidTransform.inverse)方法对刚体变换取逆：

```python

X_AB.inverse()

```

现在假设你有4个坐标系，分别为世界坐标系、A坐标系、B坐标系和C坐标系，定义如下：

-- A坐标系在世界坐标系中的表达（`X_WA`）

-- B坐标系在A坐标系中的表达（`X_AB`）

-- B坐标系在C坐标系中的表达（`X_CB`）

**请在下方指定的函数中填写代码，计算以下变换：**

(1) `X_WB`，即B坐标系在世界坐标系中的表达

(2) `X_CW`，即世界坐标系在C坐标系中的表达

In [22]:
def compute_X_WB(X_WA, X_AB, X_CB):
    """
    计算B坐标系在世界坐标系下的表达
    """
    X_WB = X_WA @ X_AB
    return X_WB


In [23]:
def compute_X_CW(X_WA, X_AB, X_CB):
    """
    计算世界坐标系在C坐标系下的表达
    """
    X_WB = X_WA @ X_AB
    X_BC = X_CB.inverse()
    X_WC = X_WB @ X_BC
    X_CW = X_WC.inverse()
    return X_CW


# 设计抓取位姿

抓取位姿通常在物体坐标系下定义，这样抓取位姿${^OX^G}$就与物体的姿态无关。抓取位姿在世界坐标系下可以通过如下公式计算：

$${{^WX^G} = {}{^W}X^{O}} {^OX^G},$$

其中$W$表示世界坐标系，$G$表示抓取器坐标系，遵循教材中的惯例。

你应该从下方的可视化中注意到，夹爪坐标系与世界坐标系不同。特别地，夹爪坐标系的+y轴竖直向下，+z轴朝向后方。这是本题的重要观察点。

**现在请根据下图中夹爪和物体的朝向，设计一个满足以下条件的抓取位姿：**

- **夹爪的位置应在世界坐标系下高于目标物体中心0.02个单位**

- **夹爪的z轴与物体的x轴对齐**

- **夹爪的x轴与物体的z轴对齐**

- **请分别在物体坐标系和世界坐标系下写出抓取位姿**

**请记住：X轴为红色，Y轴为绿色，Z轴为蓝色。**

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/grasp_pose_design.png" width="500">



In [34]:
# 建立如上图所示物体在世界坐标系下的位姿。
p0_WO = [0.5, 0.1, 0]  # 物体在世界坐标系下的位置
R0_WO = RotationMatrix.MakeZRotation(-np.pi / 2)
R0_WO = R0_WO.multiply(RotationMatrix.MakeXRotation(np.pi / 2))
X_WO = RigidTransform(R0_WO, p0_WO)


In [ ]:
def design_grasp_pose(X_WO):
    """
    设计抓取位姿
    """
    R_OG = RotationMatrix(np.array([
        [-1, 0, 0],   # x_G: 物体-x轴
        [0, -1, 0],   # y_G: 物体-y轴 (尝试正方向)
        [0, 0, 1]    # z_G: 物体z轴
    ]))
    p_OG = np.array([0, 0, 0.02])
    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @ X_OG
    return X_OG, X_WG


**完成后，请运行下方的测试单元，确保一切正常！**

In [38]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_rigid_transforms import TestRigidTransforms

Grader.grade_output([TestRigidTransforms], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 2/4.

Score for Testing X_CW is 1/1.

Score for Testing X_WB is 1/1.

Score for Testing grasp pose is 0/2.
- Test Failed: False is not true



# GRADESCOPE 验证

运行下方单元格，并将你的答案（保留4位小数）复制粘贴到Gradescope。

In [27]:
X_WA = RigidTransform([0.1, 0.2, 0.3])
X_AB = RigidTransform(
    RotationMatrix.MakeXRotation(np.pi / 2), np.array([-0.1, 0.4, 0.5])
)
X_CB = RigidTransform(
    RotationMatrix(np.array([[0, 0, 1], [0, -1, 0], [1, 0, 0]])),
    np.array([0.0, 0.0, 0.4]),
)

## Verification 1: `compute_X_WB`
**Question:** What is the z-position of frame `B` relative to frame `W`

In [28]:
print(f"{compute_X_WB(X_WA, X_AB, X_CB).GetAsMatrix4()[2, 3].item()}")

0.8


## Verification 2: `compute_X_WB`
**Question:** What is the x-position of frame `B` relative to frame `W`

In [29]:
print(f"{compute_X_CW(X_WA, X_AB, X_CB).GetAsMatrix4()[0, 3].item()}")

0.6000000000000001
